In [77]:
import sys
from pathlib import Path

# Add project root to sys.path
sys.path.append(str(Path.cwd().parent))

from methods.ipsw import estimate_sampling_scores, estimate_ate_ipsw
from methods.ipsw import IPSWEstimator
from methods.aipsw import estimate_ate_aipsw
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [78]:
def simulate_data(n=3000, seed=123):
    rng = np.random.default_rng(seed)

    # Covariates
    X1 = rng.normal(0, 1, size=n)
    X2 = rng.binomial(1, 0.5, size=n)

    # Sampling mechanism
    # Trial oversamples high X1 and high X2 individuals
    logits_S = -0.3 + 1.0 * X1 + 1.0 * X2
    pS = 1 / (1 + np.exp(-logits_S))
    S = rng.binomial(1, pS)

    # Randomized treatment in trial only
    T = np.zeros(n)
    mask_trial = S == 1
    T[mask_trial] = rng.binomial(1, 0.5, mask_trial.sum())

    # Heterogeneous treatment effect:
    # tau_i = 2 + 3 * X1
    tau_i = 2 + 3 * X1

    # Outcome model
    beta1, beta2 = 1.0, -1.0
    eps = rng.normal(0, 1, size=n)

    Y = tau_i * T + beta1 * X1 + beta2 * X2 + eps

    df = pd.DataFrame({
        "X1": X1,
        "X2": X2,
        "S": S,
        "T": T,
        "Y": Y,
        "tau_true": tau_i,
        "pS_true": pS
    })

    return df

import numpy as np

def bootstrap_ate_ipsw(
    X,
    s,
    a,
    y,
    *,
    model_cls,
    model_kwargs=None,
    proba_method="predict_proba",
    weight_type="inverse_odds",
    stabilized=True,
    clip=(0.01, 50),
    B=300,
    random_state=42,
):
    """
    Bootstrap CI for IPSW ATE using a given sampling model class (LR, XGB, RF, etc.).

    Parameters
    ----------
    model_cls : class
        A classifier class, e.g. LogisticRegression, XGBClassifier, RandomForestClassifier.
    model_kwargs : dict, optional
        Keyword args passed to model_cls(**model_kwargs).
    """

    model_kwargs = model_kwargs or {}
    rng = np.random.default_rng(random_state)
    n = len(y)
    boots = []

    # Make everything np arrays
    if hasattr(X, "iloc"):
        X_full = X  # keep as DataFrame, we'll subset with iloc
    else:
        X_full = np.asarray(X)

    s_full = np.asarray(s).ravel()
    a_full = np.asarray(a).ravel()
    y_full = np.asarray(y).ravel()

    for b in range(B):
        idx = rng.integers(0, n, n)  # resample rows with replacement

        # Subsample
        if hasattr(X_full, "iloc"):
            X_b = X_full.iloc[idx]
        else:
            X_b = X_full[idx]

        s_b = s_full[idx]
        a_b = a_full[idx]
        y_b = y_full[idx]

        # 1. Fit sampling model P(S=1|X) on bootstrap sample
        model_b = model_cls(**model_kwargs)
        ps_b, _ = estimate_sampling_scores(
            X_b,
            s_b,
            model=model_b,
            proba_method=proba_method,
        )

        # 2. IPSW ATE on bootstrap sample
        res_b = estimate_ate_ipsw(
            y=y_b,
            a=a_b,
            s=s_b,
            ps=ps_b,
            weight_type=weight_type,
            stabilized=stabilized,
            clip=clip,
        )
        boots.append(res_b.ate)

    boots = np.array(boots)
    ate_hat = float(np.mean(boots))
    ci_lower = float(np.percentile(boots, 2.5))
    ci_upper = float(np.percentile(boots, 97.5))

    return ate_hat, ci_lower, ci_upper, boots


# IPSW

In [79]:
df = simulate_data(n=8000, seed=42)

# 1. Design matrices / vectors
X = df[["X1", "X2"]]          # covariates used in sampling model
s = df["S"].to_numpy()        # sample indicator (1 = trial, 0 = target)
a = df["T"].to_numpy()        # treatment
y = df["Y"].to_numpy()        # outcome

# True ATE in the target population (S = 0)
ATE_target_true = df.loc[df["S"] == 0, "tau_true"].mean()

In [80]:
# 2. Estimate P(S = 1 | X)
ps_hat, model = estimate_sampling_scores(X, s)

# 3. Use IPSW to estimate ATE in the *target* population
# For generalizing to S=0, inverse-odds weights are standard:
result = estimate_ate_ipsw(
    y=y,
    a=a,
    s=s,
    ps=ps_hat,
    weight_type = "inverse_odds",   # key for generalizability to S=0
    stabilized=True,
    clip=(0.01, 50),              # optional, to tame extreme weights
)

"""ate_lr, ci_lr_lo, ci_lr_hi, boots_lr = bootstrap_ate_ipsw(
    X=X,
    s=s,
    a=a,
    y=y,
    model_cls=LogisticRegression,
    model_kwargs=dict(solver="lbfgs", max_iter=2000),
    weight_type="inverse_odds",   # generalize from S=1 to S=0
    stabilized=True,
    clip=(0.01, 50),
    B=300,
)

print("Logistic Regression IPSW ATE:", ate_lr)
print("95% CI (LR):", (ci_lr_lo, ci_lr_hi))"""

'ate_lr, ci_lr_lo, ci_lr_hi, boots_lr = bootstrap_ate_ipsw(\n    X=X,\n    s=s,\n    a=a,\n    y=y,\n    model_cls=LogisticRegression,\n    model_kwargs=dict(solver="lbfgs", max_iter=2000),\n    weight_type="inverse_odds",   # generalize from S=1 to S=0\n    stabilized=True,\n    clip=(0.01, 50),\n    B=300,\n)\n\nprint("Logistic Regression IPSW ATE:", ate_lr)\nprint("95% CI (LR):", (ci_lr_lo, ci_lr_hi))'

In [81]:
est_xgb = IPSWEstimator(
    model=XGBClassifier(
        max_depth=3,
        n_estimators=300,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
    ),
    weight_type="inverse_odds",   # generalizability to S = 0
    stabilized=True,
    clip=(0.01, 50),
)

est_xgb.fit(X, s)
res_xgb = est_xgb.estimate(y, a)


"""ate_xgb, ci_xgb_lo, ci_xgb_hi, boots_xgb = bootstrap_ate_ipsw(
    X=X,
    s=s,
    a=a,
    y=y,
    model_cls=XGBClassifier,
    model_kwargs=dict(
        max_depth=3,
        n_estimators=300,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="logloss",
        n_jobs=-1,
    ),
    weight_type="inverse_odds",
    stabilized=True,
    clip=(0.01, 50),
    B=300,
)

print("XGBoost IPSW ATE:", ate_xgb)
print("95% CI (XGB):", (ci_xgb_lo, ci_xgb_hi))"""

'ate_xgb, ci_xgb_lo, ci_xgb_hi, boots_xgb = bootstrap_ate_ipsw(\n    X=X,\n    s=s,\n    a=a,\n    y=y,\n    model_cls=XGBClassifier,\n    model_kwargs=dict(\n        max_depth=3,\n        n_estimators=300,\n        learning_rate=0.05,\n        subsample=0.8,\n        colsample_bytree=0.8,\n        eval_metric="logloss",\n        n_jobs=-1,\n    ),\n    weight_type="inverse_odds",\n    stabilized=True,\n    clip=(0.01, 50),\n    B=300,\n)\n\nprint("XGBoost IPSW ATE:", ate_xgb)\nprint("95% CI (XGB):", (ci_xgb_lo, ci_xgb_hi))'

In [ ]:
est_rf = IPSWEstimator(
    model=RandomForestClassifier(
        n_estimators=500,
        max_depth=6,
    ),
    weight_type="inverse_odds",  # generalizability to S = 0
    stabilized=True,
)

est_rf.fit(X, s)
res_rf = est_rf.estimate(y, a)

"""ate_rf, ci_rf_lo, ci_rf_hi, boots_rf = bootstrap_ate_ipsw(
    X=X,
    s=s,
    a=a,
    y=y,
    model_cls=RandomForestClassifier,
    model_kwargs=dict(
        n_estimators=500,
        max_depth=6,
        n_jobs=-1,
    ),
    weight_type="inverse_odds",
    stabilized=True,
    clip=(0.01, 50),
    B=300,
)

print("Random Forest IPSW ATE:", ate_rf)
print("95% CI (RF):", (ci_rf_lo, ci_rf_hi))"""


' est_rf.fit(X, s)\nres_rf = est_rf.estimate(y, a)\nate_rf, ci_rf_lo, ci_rf_hi, boots_rf = bootstrap_ate_ipsw(\n    X=X,\n    s=s,\n    a=a,\n    y=y,\n    model_cls=RandomForestClassifier,\n    model_kwargs=dict(\n        n_estimators=500,\n        max_depth=6,\n        n_jobs=-1,\n    ),\n    weight_type="inverse_odds",\n    stabilized=True,\n    clip=(0.01, 50),\n    B=300,\n)\n\nprint("Random Forest IPSW ATE:", ate_rf)\nprint("95% CI (RF):", (ci_rf_lo, ci_rf_hi))'

In [83]:
print("TRUE IPSW ATE (target):", ATE_target_true)
print("XGB IPSW ATE (target):", res_xgb.ate)
print("RF IPSW ATE (target):", res_rf.ate)
print("IPSW ATE estimate LR (target):", result.ate)

TRUE IPSW ATE (target): 0.6804878754636353
XGB IPSW ATE (target): 0.6301638712287438
RF IPSW ATE (target): 0.989809553368693
IPSW ATE estimate LR (target): 0.656005755717767


# Augmented IPSW

In [84]:
ps, _ = estimate_sampling_scores(X, s)

result = estimate_ate_aipsw(
    y=y,
    a=a,
    s=s,
    X=X,
    ps=ps,                    # or let it estimate inside
    weight_type="inverse_odds",
    stabilized=True,
    clip=(0.1, 10.0),
    target="nontrial",
)

print("AIPSW ATE:", result.ate)

AIPSW ATE: 0.6572131063034092
